*Please name your Example notebook as transparently as possible. Feel free to add a description of the workflow's goal. [Remove this line when creating your example]*

# Example Notebook Template

**Author of this template** : Michèle Masson-Trottier [Remove this line when creating your example]

**Author**: 
*Include the name of all authors of the workflow showcased in the example notebook*.

**Date**:

### Citation and Resources:
*Please include the citations for all tools, software, and publications referenced in the workflow. If you are basing your workflow on another previously published example notebook, please cite the name and DOI.*

#### Tools included in this workflow
*Cite the tools based on their preferred method (example through a Zenodo DOI or a publication)*

#### Workflows this work is based on (if applicable)
*If you started from another published workflow, please cite it here*

#### Publications
*Any publication that helped you build your example should be cited here*

#### Educational resources
*Any educational resource (for example Andy's Brain Book, FSL tutorials...) should be cited here.*

#### Dataset
*Please cite the database and dataset used in your example*

**Optional:** Add a Table of Contents (ToC) below if it helps navigation during interactive use on Neurodesk. 
When rendered on GitHub Pages, a sidebar ToC is already available, so an inline ToC may be unnecessary.

Each link must match a corresponding section heading: ` ## 1. Data Preparation → #1-Data-Preparation` (keep spaces as hyphens, preserve capital letters), as demonstrated in the next cell.


[Remove this cell when creating your example]

## Table of content
[1. Data Preparation](#1.-Data-Preparation)  
[2. Analysis](#2.-Analysis)  
[3. Results](#3.-Results) 

[Remove this cell if no ToC is used in your notebook]

## Load software tools and import python libraries

Please have a bloc for loading the different modules needed for your example. If you can not find all the required tools in Neurodesk, feel free to add tools using the web-based GUI https://www.neurodesk.org/neurocontainers-ui/

In [ ]:
#load FSL 6.0.4
import module
await module.load('fsl/6.0.4')
await module.list()

Please consult the python packages included in the base image [see list here](https://github.com/neurodesk/neurodesktop/blob/main/Dockerfile). If you are using python packages different than these, be sure to `pip install` them in the workflow.

In [ ]:
%%capture 
!pip install pydicom

In [ ]:
# Import the necessary libraries
import pydicom # Library to work with DICOM files
import matplotlib.pyplot as plt # Plotting library for displaying images
import os # Standard library to interact with the filesystem

## 1. Data Preparation
*Please indicate clearly where the data is sourced. In this template, there's an example for Datalad and OSF*

# Our code (not neurodesk template) :
# START TO RUN FROM HERE, IGNORE CELLS ABOVE

In [4]:
%gui wx
import sys
import os

#####################
# Import of utils.py functions
#####################
# Required to get utils.py and access its functions
notebook_dir = os.path.abspath("")
parent_dir = os.path.abspath(os.path.join(notebook_dir, '..'))
sys.path.append(parent_dir)
sys.path.append('.')
from utils import loadFSL, FSLeyesServer, mkdir_no_exist


#############################
# Loading fsl and freesurfer within Neurodesk
# You can find the list of available other modules by clicking on the "Softwares" tab on the left
#############################
import lmod
await lmod.purge(force=True)
await lmod.load('fsl/6.0.7.4')
await lmod.load('freesurfer/7.4.1')
await lmod.list()

####################
# Setup FSL path
####################
loadFSL()

###################
# Load all relevant libraries for the lab
##################
import fsl.wrappers
from fsl.wrappers import fslmaths

import mne_nirs
import nilearn
from nilearn.datasets import fetch_development_fmri

import mne
import mne_nirs
import dipy
from dipy.data import fetch_bundles_2_subjects, read_bundles_2_subjects
import xml.etree.ElementTree as ET
import os.path as op
import nibabel as nib
import glob

import ants

import openneuro
from mne.datasets import sample
from mne_bids import BIDSPath, read_raw_bids, print_dir_tree, make_report


# Useful imports to define the direct download function below
import requests
import urllib.request
from tqdm import tqdm


# FSL function wrappers which we will call from python directly
from fsl.wrappers import fast, bet
from fsl.wrappers.misc import fslroi
from fsl.wrappers import flirt
from fsl.wrappers import mcflirt

# General purpose imports to handle paths, files etc
import glob
import pandas as pd
import numpy as np
import json
import subprocess
import gc

Gtk-Message: 11:19:58.683: Failed to load module "canberra-gtk-module"


In [5]:
anatomical_path = op.join("subject101410", "T1w", "T1w.nii.gz") #the original brain
subject_id = '101410'

In [6]:
derivatives = op.join("derivatives", "preprocessed_data", "subject_101410")
deriv_anat = op.join(derivatives, "anat")
deriv_func = op.join(derivatives, "func")

for d in (deriv_anat, deriv_func):
    os.makedirs(d, exist_ok=True)

print("Derivatives will be written to:", os.path.abspath(derivatives))

Derivatives will be written to: /data/derivatives/preprocessed_data/subject_101410


In [7]:
#bids_root   = "/data/NX-NSSP"  
#preproc_root = op.join(bids_root, 'derivatives','preprocessed_data')
#deriv_root = op.join(bids_root, 'derivatives')

In [8]:
fsleyesDisplay = FSLeyesServer()
fsleyesDisplay.show()

11:20:03: Debug: Adding duplicate image handler for 'Windows bitmap file'
11:20:03: Debug: Adding duplicate animation handler for '1' type
11:20:03: Debug: Adding duplicate animation handler for '2' type
11:20:03: Debug: Adding duplicate image handler for 'Windows bitmap file'
11:20:03: Debug: Adding duplicate animation handler for '1' type
11:20:03: Debug: Adding duplicate animation handler for '2' type

(ipykernel_launcher.py:6063): Gtk-CRITICAL **: 11:20:03.487: gtk_window_resize: assertion 'height > 0' failed


In [9]:
fsleyesDisplay.resetOverlays()
fsleyesDisplay.load(anatomical_path)


(ipykernel_launcher.py:6063): Gdk-WARNING **: 11:20:09.802: gdkdrawable-x11.c:952 drawable is not a pixmap or window


In [ ]:
def get_skull_stripped_anatomical(deriv_anat, anatomical_path, subject_id, robust=False):
    """
    Function to perform skull-stripping (removing the skull around the brain).
    This is a simple wrapper around the brain extraction tool (BET) in FSL's suite
    It assumes data to be in the BIDS format (which we will cover in the following weeks).
    The method also saves the brain mask which was used to extract the brain.

    The brain extraction is conducted only on the T1w of the participant.

    Parameters
    ----------
    subject_id: string
        Subject ID, the subject on which brain extraction should be conducted.
    robust: bool
        Whether to conduct robust center estimation with BET or not. Default is False.
    """
    # We perform here skull stripping (you'll learn more about it next week!).
    # For now all you need to do is that we remove the bones and flesh from the MRI to get the brain!
    betted_brain_path = op.join(deriv_anat, 'sub-{}_T1w'.format(subject_id))
    os.system('bet {} {} -m {}'.format(anatomical_path, betted_brain_path, '-R' if robust else ''))
    print("Done with BET.")

resulting_mask_path = op.join(deriv_anat, 'sub-101410_T1w_mask')
get_skull_stripped_anatomical(deriv_anat, anatomical_path, subject_id, robust=True)

In [ ]:
fsleyesDisplay.load(resulting_mask_path)

In [ ]:
def apply_fsl_math_approach(img_path, mask_path, masked_img_path):
    os.system('fslmaths {} -mas {} {}'.format(img_path, mask_path, masked_img_path))
    
betted_brain_path = op.join(deriv_anat, 'sub-101410_T1w.nii.gz') # The brain without skull is in the derivatives folder
resulting_mask_path = op.join(deriv_anat, 'sub-101410_T1w_mask.nii.gz') # The mask to use

apply_fsl_math_approach(anatomical_path, resulting_mask_path, betted_brain_path)

In [ ]:
fsleyesDisplay.resetOverlays()
fsleyesDisplay.load(betted_brain_path)

## 1)b) Segmentation

In [ ]:
bet_path = op.join(deriv_anat, 'sub-101410_T1w')
fast_target = bet_path # Replace with either anatomical_path or bet_path (note: you can try both and decide which is more reasonable!)

[os.remove(f) for f in glob.glob(op.join(deriv_anat, '*fast*'))] # Just to clean the directory in between runs of the cell
segmentation_path = op.join(deriv_anat, 'sub-101410_T1w_fast')
fast(imgs=[fast_target], out=segmentation_path, n_classes=3)

pve_0 is CSF, pve_1 is grey matter, pve_2 is white matter

In [ ]:
fsleyesDisplay.resetOverlays()
fsleyesDisplay.load(bet_path)
fsleyesDisplay.load(glob.glob(op.join(deriv_anat,'*pve_0*'))[0])
fsleyesDisplay.load(glob.glob(op.join(deriv_anat,'*pve_1*'))[0])
fsleyesDisplay.load(glob.glob(op.join(deriv_anat,'*pve_2*'))[0])
fsleyesDisplay.displayCtx.getOpts(fsleyesDisplay.overlayList[1]).cmap = 'Red'
fsleyesDisplay.displayCtx.getOpts(fsleyesDisplay.overlayList[2]).cmap = 'Green'
fsleyesDisplay.displayCtx.getOpts(fsleyesDisplay.overlayList[3]).cmap = 'Blue'

## 2)a) Concatenation

In [8]:
# Create functional paths
functional_path_LR = op.join("subject101410", "fMRI", "tfMRI_MOTOR_LR", "tfMRI_MOTOR_LR.nii") 
functional_path_RL = op.join("subject101410", "fMRI", "tfMRI_MOTOR_RL", "tfMRI_MOTOR_RL.nii") 

print("Functional paths have been created")

Functional paths have been created


In [9]:
# Code to verify folder size
total_size = sum(os.path.getsize(os.path.join(deriv_func, f)) for f in os.listdir(deriv_func))
print(f"Remaining size in derivatives folder: {total_size / (1024**2):.2f} MB")


Remaining size in derivatives folder: 953.60 MB


In [10]:
###########################################
# Variance Normalisation - Run LR
###########################################
print("Normalizing variance for LR run")

# Calculate the variance for LR
variance_LR = subprocess.check_output(["fslstats", functional_path_LR, "-V"]).decode().split()[1]
print(f"Global variance for LR: {variance_LR}")

# Compute the scaling factor (1 / sqrt(variance))
scale_LR = 1 / (float(variance_LR) ** 0.5)
print(f"Scaling factor for LR: {scale_LR}")

# Apply scaling to normalize the variance
scaled_LR_path = os.path.join(deriv_func, "tfMRI_MOTOR_LR_varscaled.nii.gz")
subprocess.run(["fslmaths", functional_path_LR, "-mul", str(scale_LR), scaled_LR_path], check=True)
print(f"Saved normalized LR run: {scaled_LR_path}\n")

# Delete intermediary variables to free memory
del variance_LR, scale_LR
gc.collect()

###########################################
# Variance Normalisation - Run RL
###########################################
print("Normalizing variance for RL run")

# Calculate the variance for RL
variance_RL = subprocess.check_output(["fslstats", functional_path_RL, "-V"]).decode().split()[1]
print(f"Global variance for RL: {variance_RL}")

# Compute the scaling factor for RL (1 / sqrt(variance))
scale_RL = 1 / (float(variance_RL) ** 0.5)
print(f"Scaling factor for RL: {scale_RL}")

# Apply scaling to normalize the variance
scaled_RL_path = os.path.join(deriv_func, "tfMRI_MOTOR_RL_varscaled.nii.gz")
subprocess.run(["fslmaths", functional_path_RL, "-mul", str(scale_RL), scaled_RL_path], check=True)
print(f"Saved normalized RL run: {scaled_RL_path}\n")

# Delete intermediary variables to free memory
del variance_RL, scale_RL
gc.collect()

###########################################
# Concatenate the two runs
###########################################
print("Concatenating normalized runs...")

concat_path = os.path.join(deriv_func, f"sub-{subject_id}_motor_concat_varscaled.nii.gz")
subprocess.run(["fslmerge", "-t", concat_path, scaled_LR_path, scaled_RL_path], check=True)

print(f"Done! Final concatenated file: {concat_path}")

Normalizing variance for LR run
Global variance for LR: 519161344.000000
Scaling factor for LR: 4.388830677732065e-05
Saved normalized LR run: derivatives/preprocessed_data/subject_101410/func/tfMRI_MOTOR_LR_varscaled.nii.gz

Normalizing variance for RL run
Global variance for RL: 519202016.000000
Scaling factor for RL: 4.3886587735293315e-05
Saved normalized RL run: derivatives/preprocessed_data/subject_101410/func/tfMRI_MOTOR_RL_varscaled.nii.gz

Concatenating normalized runs...
Done! Final concatenated file: derivatives/preprocessed_data/subject_101410/func/sub-101410_motor_concat_varscaled.nii.gz


## 2)b) Motion correction

In [ ]:
print("Starting the motion correction")

path_motioncorr_LR = op.join(deriv_func, "tfMRI_MOTOR_LR_motioncorrected")            # path to the motion corrected file
path_motioncorr_RL = op.join(deriv_func, "tfMRI_MOTOR_RL_motioncorrected")            # path to the motion corrected file

mcflirt(infile=functional_path_LR, o=path_motioncorr_LR, plots=True, report=True, dof=6, mats=True)
print("Motor LR corrected")
mcflirt(infile=functional_path_RL, o=path_motioncorr_RL, plots=True, report=True, dof=6, mats=True)
print("Motor RL corrected")

# Display result of LR
fsleyesDisplay.resetOverlays()
fsleyesDisplay.load(functional_path_LR)
fsleyesDisplay.load(path_motioncorr_LR)
print("Motion corrected done")

In [ ]:
# Cell to run if save is not possible anymore ### ATTENTION SUPPRIME TOOOOUT LES PREPROCESS DATA!!!!!!!!!!!
 
# import shutil
# import os

# folder_to_clear = "derivatives/preprocessed_data/subject_101410"
# if os.path.exists(folder_to_clear):
#     shutil.rmtree(folder_to_clear)
#     print("Dossier supprimé pour libérer de l'espace.")

# In order to determine how many in how many images the subject moves more than what is tolerated, we can run the cell below

In [ ]:
import matplotlib.pyplot as plt

lr_par_path = op.join(deriv_func, 'tfMRI_MOTOR_LR_motioncorrected.par')
rl_par_path = op.join(deriv_func, 'tfMRI_MOTOR_RL_motioncorrected.par')

def load_mot_params_fsl_6_dof(path):
    return pd.read_csv(path, sep='  ', header=None, engine='python', names=['Rotation x', 'Rotation y', 'Rotation z','Translation x', 'Translation y', 'Translation z'])

mot_params_LR = load_mot_params_fsl_6_dof(lr_par_path)
mot_params_LR

mot_params_RL = load_mot_params_fsl_6_dof(rl_par_path)
mot_params_RL

def compute_FD_power(mot_params):
    framewise_diff = mot_params.diff().iloc[1:]

    rotation_params = framewise_diff[['Rotation x', 'Rotation y', 'Rotation z']]
    
    convert_rot_to_arc = rotation_params*50                                    # transform from angle to arc of circle for a radius of 50mm, normal used values are 50 or 90 mm
    translation_params = framewise_diff[['Translation x', 'Translation y', 'Translation z']]
    fd = convert_rot_to_arc.abs().sum(axis=1) + translation_params.abs().sum(axis=1)
    return fd

fd_LR = compute_FD_power(mot_params_LR).to_numpy()
threshold_LR = np.quantile(fd_LR,0.75) + 1.5*(np.quantile(fd_LR,0.75) - np.quantile(fd_LR,0.25))

#%matplotlib inline
plt.plot(list(range(1, fd_LR.size+1)), fd_LR)
plt.xlabel('Volume')
plt.ylabel('FD displacement (mm)')
plt.hlines(threshold_LR, 0, 370,colors='black', linestyles='dashed', label='FD threshold')
plt.legend()
plt.show()

print(f"Indices of frames of LR motion above threshold: {(np.where(fd_LR > threshold_LR)[0] + 1).tolist()}")

fd_RL = compute_FD_power(mot_params_RL).to_numpy()
threshold_RL = np.quantile(fd_RL,0.75) + 1.5*(np.quantile(fd_RL,0.75) - np.quantile(fd_RL,0.25))

plt.plot(list(range(1, fd_RL.size+1)), fd_RL)
plt.xlabel('Volume')
plt.ylabel('FD displacement (mm)')
plt.hlines(threshold_RL, 0, 370,colors='black', linestyles='dashed', label='FD threshold')
plt.legend()
plt.show()

print(f"Indices of frames of RL motion above threshold: {(np.where(fd_RL > threshold_RL)[0] + 1).tolist()}")

## 2)c) Co-registration (bonus)

In [10]:
import nibabel as nib

In [11]:
betted_brain_path = op.join(deriv_anat, 'sub-{}_T1w'.format(subject_id))
concat_path = os.path.join(deriv_func, f"sub-{subject_id}_motor_concat_varscaled.nii.gz")

reference= betted_brain_path
source= nib.load(concat_path)
data_source = source.get_fdata()
print("Dimensions :", data_source.shape)

volume_index = 10  # par exemple, le 11e volume
volume_source = data_source[..., volume_index]
print(volume_source.shape) 

#mni_template = op.expandvars(op.join('$FSLDIR', 'data', 'standard', 'MNI152_T1_1mm_brain'))
#reference= mni_template

spacing = tuple(source.header.get_zooms()[:3])
origin = tuple(source.affine[:3, 3])
moving_image = ants.from_numpy(volume_source, origin=origin, spacing=spacing)
fixed_image = ants.image_read(reference + '.nii.gz')

# Compute the transformation (non linear) to put align the moving image to the fixed image
transformation = ants.registration(fixed=fixed_image, moving=moving_image, type_of_transform = 'SyN' )

# After the transformation has been computed, apply it
warpedImage = ants.apply_transforms(fixed=fixed_image, moving=moving_image, transformlist=transformation['fwdtransforms'])

# Save the image to disk
resultAnts = op.join(deriv_anat,'sub-{}_T1w_mni_SyN.nii.gz'.format(subject_id))
ants.image_write(warpedImage, resultAnts)

# Inspect the results with FSLeyes or freeview, as you prefer :)

Dimensions : (91, 109, 91, 568)
(91, 109, 91)


In [12]:
fsleyesDisplay.resetOverlays()
fsleyesDisplay.load(reference) 
fsleyesDisplay.load(resultAnts)

## 2)d) Gaussian smoothing

In [13]:

output_path = concat_path
cmd = 'fslmaths {} -s {} {}_smoothed-6mm'.format(output_path, 6/2.3548, output_path)
subprocess.run(['fslmaths',output_path, '-s', str(6/2.3548), '{}_smoothed-6mm'.format(output_path)])
results_smoothing = '{}_smoothed-6mm.nii.gz'.format(output_path)


In [14]:
fsleyesDisplay.resetOverlays()
fsleyesDisplay.load(output_path) 
fsleyesDisplay.load(results_smoothing)

## 3)

In [ ]:
from nilearn.glm.first_level import make_first_level_design_matrix, FirstLevelModel
from nilearn.plotting import plot_design_matrix
from nilearn.plotting import plot_stat_map
from IPython.display import display
from nilearn.image import mean_img
import nibabel as nib

In [ ]:
fMRI_path = '/data/subject101410/fMRI'

func_path_LR = op.join(fMRI_path, 'tfMRI_MOTOR_LR') 
func_path_RL = op.join(fMRI_path, 'tfMRI_MOTOR_RL') 

events_LR = pd.read_csv(op.join(func_path_LR, 'events_LR.csv'), sep=',')
events_RL = pd.read_csv(op.join(func_path_RL, 'events_RL.csv'), sep=',')

print("Events Table — LR Run")
pd.set_option('display.float_format', '{:,.2f}'.format)
display(events_LR.head(10))

print("Events Table — RL Run")
pd.set_option('display.float_format', '{:,.2f}'.format)
display(events_RL.head(10))


In [ ]:
import nibabel as nib
img = nib.load(func_img)
n_scans = img.shape[3]           # number of volumes
TR = img.header.get_zooms()[3]   # repetition time
frame_times = np.arange(n_scans) * TR  # time vector for each volume

# design matrix LR
img_LR = nib.load(func_img_LR) #Replace func_img_LR by the right file
n_scans_LR = img_LR.shape[3]
TR = 0.72
frame_times_LR = np.arange(n_scans_LR) * TR

design_matrix_LR = make_first_level_design_matrix(
    frame_times_LR,
    events_LR,
    hrf_model='spm',
    drift_model='cosine',
    high_pass=0.01)

print('Design Matrix LR')
plot_design_matrix(fmri_glm_LR.design_matrices_[0])

# design matrix RL
img_RL = nib.load(func_img_RL) #Replace func_img_LR by the right file
n_scans_RL = img_RL.shape[3]
frame_times_RL = np.arange(n_scans_RL) * TR

design_matrix_RL = make_first_level_design_matrix(
    frame_times_RL,
    events_RL,
    hrf_model='spm',
    drift_model='cosine',
    high_pass=0.01)

print( 'Design Matrix RL')
plot_design_matrix(fmri_glm_RL.design_matrices_[0])
plt.show()

# 4)

In [ ]:
GLM_fMRI = FirstLevelModel(t_r=0.72,
                           noise_model='ar1',
                           standardize=False,
                           hrf_model='spm',
                           drift_model=None,
                           high_pass=.01)

# Fit the model to our design and data
fmri_glm_LR = GLM_fMRI.fit(fmri_img, events_LR)
fmri_glm_RL = GLM_fMRI.fit(fmri_img, events_RL)

# Get regressors
regressors_LR = fmri_glm_LR.design_matrices_[0].columns.tolist()
print(regressors_LR)
regressors_RL = fmri_glm_LR.design_matrices_[0].columns.tolist()
print(regressors_RL)

# Compute and plot t-map for each regressor and run
for reg in regressors_LR:
    t_map = fmri_glm.compute_contrast(reg, output_type='stat')
    plotting.plot_stat_map(
        t_map,
        threshold=3.0,
        display_mode='ortho',
        title=f'Statistical map: {reg}'
    )

for reg in regressors_RL:
    t_map = fmri_glm_RL.compute_contrast(reg, output_type='stat')
    plotting.plot_stat_map(
        t_map,
        threshold=3.0,
        display_mode='ortho',
        title=f'Statistical map: {reg}'
    )

# Comments on t-map

# 5)

In [ ]:
contrast_vector_hand_vs_foot = [1, 1, -1, -1, 0, 0] # depend on the regressor list, might be changed

activation_map_LR = fmri_glm.compute_contrast(contrast_vector_hand_vs_foot,
                                  output_type='effect_size')
activation_map_RL = fmri_glm.compute_contrast(contrast_vector_hand_vs_foot,
                                  output_type='effect_size')
#nib.save(z_map, 'z_map_uncorrected.nii.gz')

# activation map for LR
plotting.plot_stat_map(
    activation_map_LR,
    threshold=0.5,          # might need to be adjusted as needed
    display_mode='ortho',
    cut_coords=(0, 0, 0),   # might need to be adjusted as needed
    title='Hand > Foot Activation Map, LR'
))

# activation map for RL
plotting.plot_stat_map(
    activation_map_RL,
    threshold=0.5,          # might need to be adjusted as needed
    display_mode='ortho',
    cut_coords=(0, 0, 0),   # might need to be adjusted as needed
    title='Hand > Foot Activation Map, RL'
)

plt.show()

#nib.save(activation_map, 'activation_map_hand_vs_foot.nii.gz')


# END OF OUR CODE, BELOW IS THE NEURODESK TEMPLATE (IGNORE BELOW)

### Datalad

In [ ]:
%%bash

# Install the dataset from GitHub using DataLad. 
# Replace the URL with the one corresponding to the OpenNeuro dataset you want to use.
datalad install https://github.com/OpenNeuroDatasets/ds000102.git 

# Navigate into the dataset directory. 
# This should match the name of the dataset repository.
cd ds000102

# Optional: List available subjects
ls -d sub-*

# Download data for a specific subject (e.g., sub-08).
# You can change the subject ID to the one you're interested in.
datalad get sub-08

### OSF

In [ ]:
# Use the OSF client to clone the contents of the OSF project with ID 'ru43c' into the current directory
# The output is suppressed for cleaner display using > /dev/null 2>&1
!osf -p ru43c clone . > /dev/null 2>&1

# Extract the contents of the downloaded tar archive, which contains DICOM files
!tar xf osfstorage/dicoms-unsorted.tar

# Remove the now-unneeded 'osfstorage' directory to keep things tidy
!rm -rf osfstorage/

# Display the top of the folder tree for the extracted DICOM files (limited with `head` to avoid overwhelming output)
!tree dicoms-unsorted | head

# Print a count of how many unsorted DICOM files were extracted
!echo -e "...\nThere are `ls dicoms-unsorted | wc -l` unsorted DICOMs in ./dicoms-unsorted/"

### Visualisation of a dicom (one method for example)

In [ ]:
# Define the path to a DICOM file
# You can change this to any other file in the dicoms-unsorted/ directory
dcm_path = "./dicoms-unsorted/MR.1.2.276.0.7230010.3.1.3.0.4480.1677042698.772469.84.dcm"

# Read the DICOM file into a Python object
ds = pydicom.dcmread(dcm_path)

# ----------------------------
# Inspecting DICOM metadata (optional)
# ----------------------------

# Print a few key metadata fields (can be useful for QC or orientation)
print("Patient ID:", ds.get("PatientID", "N/A"))
print("Modality:", ds.get("Modality", "N/A"))
print("Study Date:", ds.get("StudyDate", "N/A"))

# ----------------------------
# Visualizing the DICOM image
# ----------------------------

# Display the image stored in the DICOM file using matplotlib
# Note: Some DICOM files (e.g., non-image series) may not contain pixel data
plt.imshow(ds.pixel_array, cmap=plt.cm.gray) # Use grayscale color map
plt.title("DICOM Slice: MR.1.1.dcm") # Title of the plot
plt.axis("off") # Hide axis ticks/labels for cleaner view
plt.show()

## 2. Analysis

*Be sure to annotate your example. You can also add Markdown blocs to provide explanations.*

In [ ]:
!bet ./ds000102/sub-08/anat/sub-08_T1w.nii.gz sub-08_T1w_brain.nii.gz

## 3. Results

In [ ]:
from ipyniivue import NiiVue
nv = NiiVue()
nv.load_volumes([{"path": "sub-08_T1w_brain.nii.gz"}])
nv

## Dependencies in Jupyter/Python
- Using the package [watermark](https://github.com/rasbt/watermark) to document system environment and software versions used in this notebook

In [ ]:
%%capture
!pip install watermark

In [ ]:
%load_ext watermark

%watermark
%watermark --iversions